In [9]:
import pandas as pd
import numpy as np

import time
import copy

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# from sklearn.cluster import Birch
# import matplotlib.pyplot as plt




In [10]:
def k_means_model_with_best_sil_score(X, random_seed = 0, kmax = 10):
  # sil = []

  # for k in range(2, kmax+1):
  #   kmeans = KMeans(n_clusters = k, random_state=random_seed).fit(X)
  #   sil.append(silhouette_score(X, kmeans.labels_, metric = 'euclidean'))

  # print(sil)
  # return sil.index(max(sil))+2

  highest_score = 0
  k = 0
  k_means_model = None

  for ks in range(2, kmax+1):
    kmeans = KMeans(n_clusters = ks, random_state=random_seed).fit(X)
    cur_score = silhouette_score(X, kmeans.labels_, metric = 'euclidean')
    if cur_score > highest_score:
      highest_score = cur_score
      k = ks
      k_means_model = kmeans

  return {"k_means_model": k_means_model, "best_k": k}

In [11]:
# data structure
class node:
    def __init__(self, centre, index, up_lv = None, down_lv = None, far = 0):
        self.centre = centre # cluster centre (vector)
        self.index = index # cashe of search result, first 100? result close to centre (list of index)
        self.up_lv = up_lv # super-cluster (node)
        self.down_lv = down_lv # sub-cluster (lsit of node)
        self.far = far # farest data in the cluster ( a boundary of the cluster) (value)

# graph tree (cluster)
class graph_tree:
    def __init__(self, X):
        self.X = X
        self.root = node(np.mean(X, axis=0), np.array([i for i in range(len(X))]))
        self.tree_deep = 0
        print("root node is created")

    def sim(self, vector_1, vector_2):
        return sum((vector_1-vector_2)**2)

    def search(self, attribute, deep = 2):
        # print("searching")
        cur = self.root
        
        while cur.down_lv:
            old_cur = cur
            cur_centre_distance = self.sim(attribute, cur.centre)
            # print(cur_centre_distance)

            closest = cur_centre_distance

            for node in cur.down_lv:
                if self.sim(attribute, node.centre) < closest:
                    closest = self.sim(attribute, node.centre)
                    cur = node


            # search_node_queue = [cur]
            # # normal search with only center counting
            # for layer in range(deep):
            #     print(search_node_queue)
            #     size = len(search_node_queue)
            #     for i in range(size):
            #         searching_node = search_node_queue.pop(0)
            #         if searching_node.down_lv:
            #             for node in searching_node.down_lv:
            #                 if self.sim(attribute, node.centre) < closest:
            #                     closest = self.sim(attribute, node.centre)
            #                     cur = node
            #                 # add down lv node into the queue
            #                 if node.down_lv:
            #                     search_node_queue += node.down_lv
                            

            # boundary search by considering the cluster boundary (far attr)
            if cur == old_cur:
                for node in cur.down_lv:
                    if self.sim(attribute, node.centre)-cur.far < closest:
                        closest = self.sim(attribute, node.centre)
                        cur = node
            
            
            if cur == old_cur:
                break

        return cur
        
    def build_tree(self):
        queue = []
        queue.append(self.root)
        
        lv = 0
    
        while queue:
            print("queue", queue)
            size = len(queue)
            print(f"\n---------------------------------- level {lv} no of node {size} ----------------------------------------")
            for node_in_this_lv in range(size):
                print(f"\nhandling node {node_in_this_lv}, info:")
                # k means required parameter and train the model
                cur = queue.pop(0) # get cur node
                cur_X = self.X[cur.index] # get cur data by index
                k_means_model = k_means_model_with_best_sil_score(cur_X) # get kmeans model

                print(f"cur node {cur}")
                print(f"no. of cur node data {len(cur_X)}")
                print("optimal K", k_means_model["best_k"], "\n")

                cur_kmeans = k_means_model["k_means_model"] # best k means model

                new_down_lv = []
                for groups in range(k_means_model["best_k"]):
                    print(f"creating node {groups}")
                    new_index_of_index = np.where(cur_kmeans.labels_ == groups)[0]

                    # calculate the farest distance
                    data_in_this_gp  = self.X[cur.index[new_index_of_index]]
                    farest_distance = np.max(np.sum((data_in_this_gp - cur_kmeans.cluster_centers_[groups])**2, axis=1))

                    new_node = node(cur_kmeans.cluster_centers_[groups], cur.index[new_index_of_index], up_lv=cur, far=farest_distance)

                    new_down_lv.append(new_node)
                    print(f"new node {new_node}")
                    # print("new node centre", new_node.centre)
                    print(f"no. of new node data {len(new_node.index)}")
                    print(f"down lv of this node {new_down_lv}")
                    
                    # least no. of data
                    if len(new_index_of_index) >= 185:
                        queue.append(new_node)
                
                cur.down_lv = new_down_lv
                

            lv += 1

            self.tree_deep = lv+1

        return 0
    
    # def print_tree(self, level = None):
    #     level = level if level else self.tree_deep
    #     queue = [self.root]
    #     for t in range(level):






In [12]:
raw = pd.read_csv(r'Data\features_30_sec.csv')
raw

random_seed = 42

In [13]:
data = raw.iloc[:,1:-1]
data
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

y = raw.iloc[:,-1]

In [14]:
test_tree = graph_tree(normalized_data)
# test_tree.show_root()
test_tree.build_tree()

root node is created
queue [<__main__.node object at 0x0000026E82B90590>]

---------------------------------- level 0 no of node 1 ----------------------------------------

handling node 0, info:
cur node <__main__.node object at 0x0000026E82B90590>
no. of cur node data 1000
optimal K 3 

creating node 0
new node <__main__.node object at 0x0000026ED3148F90>
no. of new node data 353
down lv of this node [<__main__.node object at 0x0000026ED3148F90>]
creating node 1
new node <__main__.node object at 0x0000026E82B9C950>
no. of new node data 318
down lv of this node [<__main__.node object at 0x0000026ED3148F90>, <__main__.node object at 0x0000026E82B9C950>]
creating node 2
new node <__main__.node object at 0x0000026E82B93290>
no. of new node data 329
down lv of this node [<__main__.node object at 0x0000026ED3148F90>, <__main__.node object at 0x0000026E82B9C950>, <__main__.node object at 0x0000026E82B93290>]
queue [<__main__.node object at 0x0000026ED3148F90>, <__main__.node object at 0x000

0

In [15]:
print(normalized_data.shape)
print(np.mean(normalized_data[:100], axis=0))

print(test_tree.root.centre)

(1000, 58)
[0.11348684 0.34466932 0.72605349 0.33305741 0.10877244 0.29947823
 0.09304523 0.39569181 0.15035941 0.36422083 0.17241909 0.22643159
 0.05545706 0.62273814 0.15774551 0.50904565 0.09744436 0.36549813
 0.65222521 0.12414806 0.59314374 0.13376623 0.56282273 0.16902846
 0.65715205 0.21199967 0.50119821 0.17264277 0.68976559 0.2116002
 0.40642335 0.28073522 0.52519319 0.19397631 0.37807936 0.22873911
 0.51505091 0.19394156 0.40249091 0.10072538 0.49799014 0.18860505
 0.47996194 0.18586443 0.45119759 0.10657799 0.3583135  0.13427742
 0.54500148 0.14372898 0.34880168 0.13019081 0.38442208 0.16402178
 0.44870923 0.14583445 0.48545943 0.12517942]
[0.12846951 0.42042582 0.65744286 0.31997514 0.11010242 0.42216161
 0.15245649 0.51481278 0.18463802 0.48210799 0.21161387 0.32386061
 0.10260274 0.62498145 0.14590304 0.5221195  0.11949048 0.35862408
 0.68609031 0.12730489 0.51920421 0.15606796 0.55235925 0.14981778
 0.5533549  0.21443681 0.5366843  0.13712389 0.58499709 0.21667654
 0.507

In [16]:
from collections import Counter

"""
create a new index (something like tf-idf)
data reduction rate
vs
accurate
"""
def score(search_data, acc, smoothing = 0.001, lam = 0.5):
    return lam*(acc/100)+(1-lam)*(1-(search_data/(1000+smoothing)))

my_score = []
normal_kmeans_score = []

tmp = [i*100 for i in range(10)]
for i in tmp:
    b = i
    u = i+100
    # print(normalized_data[b:u])
    print(f"-- mine result {i}")
    seaching = np.mean(normalized_data[b:u], axis=0)
    result = test_tree.search(seaching)
    
    print("search number: ", len(result.index))
    right = np.where((result.index>=b) & (result.index<u))[0] # return index
    print("hitted right result:", len(right))
    my_score.append(score(len(result.index), len(right)))

    ############
    print("-- kmeans result")
    checking = KMeans(n_clusters=10, random_state=0).fit(normalized_data)

    counts1 = Counter(checking.labels_)
    print(counts1)

    counts2 = Counter(checking.labels_[b:u])
    print(counts2)
    belongs_group = counts2.most_common()[0][0]
    print("belongs to group: ", belongs_group, "\n")
    # print(counts1.get(belongs_group), counts2.get(belongs_group))
    normal_kmeans_score.append(score(counts1.get(belongs_group), counts2.get(belongs_group)))

    

-- mine result 0
search number:  1000
hitted right result: 100
-- kmeans result
Counter({5: 219, 9: 126, 0: 125, 3: 120, 2: 109, 4: 84, 6: 69, 8: 66, 1: 51, 7: 31})
Counter({5: 30, 9: 22, 0: 20, 7: 18, 4: 8, 6: 2})
belongs to group:  5 

-- mine result 100
search number:  170
hitted right result: 95
-- kmeans result
Counter({5: 219, 9: 126, 0: 125, 3: 120, 2: 109, 4: 84, 6: 69, 8: 66, 1: 51, 7: 31})
Counter({6: 49, 9: 47, 0: 2, 3: 1, 2: 1})
belongs to group:  6 

-- mine result 200
search number:  1000
hitted right result: 100
-- kmeans result
Counter({5: 219, 9: 126, 0: 125, 3: 120, 2: 109, 4: 84, 6: 69, 8: 66, 1: 51, 7: 31})
Counter({0: 28, 2: 19, 9: 19, 3: 14, 7: 8, 5: 7, 4: 5})
belongs to group:  0 

-- mine result 300
search number:  1000
hitted right result: 100
-- kmeans result
Counter({5: 219, 9: 126, 0: 125, 3: 120, 2: 109, 4: 84, 6: 69, 8: 66, 1: 51, 7: 31})
Counter({3: 33, 5: 32, 0: 13, 2: 11, 4: 8, 8: 2, 6: 1})
belongs to group:  3 

-- mine result 400
search number:  1000


In [17]:
print(my_score)
print(normal_kmeans_score)

print(sum(my_score))
print(sum(normal_kmeans_score))

[0.5000004999995, 0.890000084999915, 0.5000004999995, 0.5000004999995, 0.5000004999995, 0.685000084999915, 0.8245000704999295, 0.7125000574999425, 0.5000004999995, 0.5000004999995]
[0.5405001094998905, 0.7105000344999655, 0.5775000624999376, 0.60500005999994, 0.5055001094998905, 0.587000062999937, 0.8405001094998905, 0.6345000254999745, 0.6380000419999581, 0.5555001094998905]
6.112003297996701
6.1945007254992746
